In [314]:
# Include Libraries
import numpy as np
import tensorflow as tf
import gym
import tensorflow.contrib.layers as layers
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import policies
import value_functions as vfuncs
import utils_pg as utils

# Environment setup
#env = "CartPole-v0"
#env="InvertedPendulum-v2"
#env = "Hopper-v2"
env="HalfCheetah-v2"

# discrete = isinstance(env.action_space, gym.spaces.Discrete)
# observation_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n if discrete else env.action_space.shape[0]
max_ep_len = 1000
num_traj = 3
#traj_length = max_ep_len*(observation_dim + 2)
latent_size = 25
use_baseline = True



In [315]:
# # Feed forward network (multi-layer-perceptron, or mlp)
# def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):
#     '''
#     Build a feed forward network
#     '''
#     Input = mlp_input
#     with tf.variable_scope(scope):
#         # Dense Layers
#         for i in range(n_layers-1):
#             dense = tf.layers.dense(inputs = Input, units = size, activation = tf.nn.relu, bias_initializer=tf.constant_initializer(1.0))
#             Input = dense
#         # Fully Connected Layer
#         out = layers.fully_connected(inputs = Input, num_outputs = output_size, activation_fn=output_activation)
#     return out

def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):

        hidden1 = layers.fully_connected(mlp_input,
                num_outputs=32,
                weights_initializer=layers.xavier_initializer(uniform=True),
                activation_fn=tf.nn.relu)
        hidden2 = layers.fully_connected(hidden1,
                num_outputs=32,
                weights_initializer=layers.xavier_initializer(uniform=True),
                activation_fn=tf.nn.relu)
        mean_na = layers.fully_connected(hidden2,
                num_outputs=output_size,
                weights_initializer=layers.xavier_initializer(uniform=True),
                activation_fn=None)

        return mean_na 

In [319]:
class MetaLearner():
    def __init__(self, env, max_ep_len, num_traj,latent_size ):
        self.env = gym.make(env)
        self.discrete = isinstance(self.env.action_space, gym.spaces.Discrete)
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n if self.discrete else self.env.action_space.shape[0]
        self.max_ep_len = max_ep_len
        self.num_traj = num_traj
        self.traj_length = self.max_ep_len*(self.observation_dim + 1 + self.env.action_space.shape[0]) # TO Change
        self.use_baseline = True
        self.latent_size = latent_size
        self.feature_size = self.observation_dim + 1 + self.env.action_space.shape[0] # HC
        self.lr = 3e-3
        self.num_layers = 1
        self.layers_size = 16
        self.num_mdps = 10
        self.model = self.env.env.model
        self.sim = MjSim(self.model)
        self.gamma = .95
        self.decoder_len = 16

        # build model
        #self.ConstructGraph()
    
    def add_placeholders(self):
        self.observation_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        if(self.discrete):
            self.action_placeholder_explore = tf.placeholder(tf.int32, shape=(None))
            self.action_placeholder_exploit = tf.placeholder(tf.int32, shape=(None))
        else:
            self.action_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.action_dim))
            self.action_placeholder_exploit= tf.placeholder(tf.float32, shape=(None,self.action_dim))
        
        ##########################
        self.n     = tf.shape(self.observation_placeholder_explore)[0] 
        self.oldlogstd_a  = tf.placeholder(name="oldlogstd", shape=[self.action_dim], dtype=tf.float32)
        ##########################
        
        self.baseline_target_placeholder = tf.placeholder(tf.float32, shape= None)
        self.advantage_placeholder_explore = tf.placeholder(tf.float32, shape=(None))
        
        
        #self.encoder_input_placeholder = tf.placeholder(tf.float32, shape= (self.num_traj,self.traj_length))
        self.encoder_input_placeholder = tf.placeholder(tf.float32, [None, None, self.feature_size])
        self.decoder_input_placeholder = tf.placeholder(tf.float32, shape= (1,self.latent_size))
        self.sequence_length_placeholder = tf.placeholder(tf.int32, [None, ])
        
        self.observation_placeholder_exploit = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        #TODO
        self.advantage_placeholder_exploit = tf.placeholder(tf.float32, shape=(None))
        
        
    def build_policy_explore(self, scope = "policy_explore"):
        if (self.discrete):
            self.action_logits = build_mlp(self.observation_placeholder_explore,self.action_dim,scope = scope,n_layers=self.num_layers,size = self.layers_size,output_activation=None)
            self.explore_action = tf.multinomial(self.action_logits,1)
            self.explore_action = tf.squeeze(self.explore_action, axis=1)
            self.explore_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.action_logits, labels = self.action_placeholder_explore)

        else:   
            action_means = build_mlp(self.observation_placeholder_explore,self.action_dim,scope,n_layers=self.num_layers, size = self.layers_size,output_activation=None)
            
            self.logstd_a     = tf.get_variable("logstd", [self.action_dim], initializer=tf.zeros_initializer())
            self.logstd_na    = tf.ones(shape=(self.n,self.action_dim), dtype=tf.float32) * self.logstd_a
            self.oldlogstd_na = tf.ones(shape=(self.n,self.action_dim), dtype=tf.float32) * self.oldlogstd_a
            
            self.explore_logprob  = utils.gauss_log_prob(mu=action_means, logstd=self.logstd_na, x=self.action_placeholder_explore)
            self.explore_action =   action_means + tf.multiply(tf.exp(self.logstd_na),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            
#             init = tf.constant(np.random.rand(1, 2))
#             log_std = tf.get_variable("log_std", [self.action_dim])
#             self.explore_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
#             mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
#             self.explore_logprob =  mvn.log_prob(value = self.action_placeholder_explore, name='log_prob')
    
    
    def build_policy_exploit(self, scope = "policy_exploit"):
        if(self.discrete):
            #self.exploit_action_logits = (tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W1) + self.d_B1), self.d_W2) + self.d_B2),self.d_W3) + self.d_B3)
            self.exploit_action_logits = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            self.exploit_action = tf.multinomial(self.exploit_action_logits,1)
            self.exploit_action = tf.squeeze(self.exploit_action, axis=1)
            self.exploit_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.exploit_action_logits, labels = self.action_placeholder_exploit)
        else:
            
            action_means = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            init = tf.constant(np.random.rand(1, 2))
            log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
            self.exploit_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
            self.exploit_logprob =  mvn.log_prob(value = self.action_placeholder_exploit, name='exploit_log_prob')

        #self.loss_grads_exploit = self.exploit_logprob * self.advantage_placeholder_exploit
        
    def NNEncoder(self, scope = "NNEncoder"):
        self.Z = build_mlp(self.encoder_input_placeholder,self.latent_size,scope = scope,n_layers=3,size = 60,output_activation=None)
    
 

    # input [num_traj, length, features (obs + action + reward) ]
    def LSTMEncoder(self, scope = "LSTMEncoder"):
        self.hidden_size = 64
        initializer = tf.random_uniform_initializer(-1, 1)
        cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, self.feature_size, initializer=initializer)
        cell_out = tf.contrib.rnn.OutputProjectionWrapper(cell, self.latent_size)
        self.output, _ = tf.nn.dynamic_rnn(cell_out,self.encoder_input_placeholder,self.sequence_length_placeholder,dtype=tf.float32,)
        batch_size = tf.shape(self.output)[0]
        max_length = tf.shape(self.output)[1]
        out_size = int(self.output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (self.sequence_length_placeholder - 1)
        flat = tf.reshape(self.output, [-1, out_size])
        self.Z = tf.reduce_mean(tf.gather(flat, index), axis = 0)
        
    
    def Decoder(self, decoder_out_dim, scope):
        return build_mlp(self.decoder_input_placeholder,decoder_out_dim,scope = scope,n_layers=1,size = 64,output_activation=None)
    
    
    def sample_paths_explore(self, env,mdp_num,Test = False, num_episodes = None):
        paths = []
        self.length = []
        episode_rewards = []
        #self.sim.model.opt.gravity[-1] = -5 - mdp_num
        for i in range(self.num_traj):
            pad = False
            state = env.reset()
            new_states,states,new_actions, actions, rewards,new_rewards = [], [], [], [], [],[]
            episode_reward = 0
            for step in range(self.max_ep_len):
                if (pad):
                    states.append([0]*self.observation_dim)
                    if(self.discrete):
                        actions.append(0)
                    else:
                        actions.append([0]*self.action_dim)
                    rewards.append(0)
                else:
                    states.append(state)
                    new_states.append(state)
                    action = self.sess.run(self.explore_action, feed_dict={self.observation_placeholder_explore : states[-1][None]})[0]
                    #action = self.policyfn.sample_action(state) #NEW
                    state, reward, done, info = env.step(action)
                    actions.append(action)
                    new_actions.append(action)
                    rewards.append(reward)
                    new_rewards.append(reward)
                    episode_reward += reward
                    if (done or step == self.max_ep_len-1):
                        self.length.append(step + 1)
                        pad = True 
                        
            episode_rewards.append(episode_reward)            
            #print("explore",np.array(actions))
            path = {"new_obs" : np.array(new_states),"observation" : np.array(states),
                                "reward" : np.array(rewards),"new_acs" : np.array(new_actions),
                                "action" : np.array(actions),"new_rewards":np.array(new_rewards)}
            paths.append(path)
        return paths, episode_rewards
    
    def sample_paths_exploit(self, env,Z,mdp_num,Test = False, num_episodes = None):
        #self.sim.model.opt.gravity[-1] = -5 - mdp_num
        
        paths = []
        num = 0
        episode_rewards = []
        for i in range(self.num_traj):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0
            for step in range(self.max_ep_len):
                states.append(state)
                action = self.sess.run(self.exploit_action, feed_dict={self.observation_placeholder_exploit : state[None], self.decoder_input_placeholder: Z})[0]
                state, reward, done, info = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
            #print("exploit",np.array(actions))
            path = {"observation" : np.array(states),
                                "reward" : np.array(rewards),
                                "action" : np.array(actions)}
            paths.append(path)
 
        #print("exploit success: ", num)
        return paths, episode_rewards
    
    def get_returns(self,paths, explore = False):
        all_returns = []
        m = 0
        for path in paths:
            rewards = path["reward"]
            returns = []
            if(explore):
                length = self.length[m]
            else:
                length = len(rewards)
            for i in range(length):
                path_returns = 0
                k = 0
                for j in range(i,length):
                    path_returns = path_returns + rewards[j]*(self.gamma)**k
                    k = k+1
                returns.append(path_returns)
            all_returns.append(returns)
            m+=1
        returns = np.concatenate(all_returns)
        return returns
    
    def stack_trajectories(self,paths):
        trajectories = []
        for path in paths:
            rewards = path["reward"]
            states = path["observation"]
            action = path["action"]
            
            SAR = []
            for i in range(len(states)):
                SAR.append(list(states[i]) + list(action[i]) + [rewards[i]])
            trajectories.append(SAR)
        return np.array(trajectories)
    
    def addBaseline(self):
        self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=self.num_layers, size = self.layers_size,output_activation=None)
        self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
        baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
        self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)

    def calculate_advantage(self,returns, observations):
        if (self.use_baseline):
            baseline = self.sess.run(self.baseline, {input_placeholder:observations})
            adv = returns - baseline
            adv = (adv - np.mean(adv))/np.std(adv)
        else:
            adv = returns
        return adv
    

        
    def ConstructGraph(self):
        tf.reset_default_graph()
        self.add_placeholders()
        
        self.build_policy_explore()
        self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
        
        #self.vf = vfuncs.NnValueFunction(session=self.sess,ob_dim=self.observation_dim)

        
        
#         self.add_placeholders()
        
#         self.build_policy_explore()
#         self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
#         self.loss_grads_explore = -self.explore_logprob * self.advantage_placeholder_explore
#         self.tvars_explore = tf.trainable_variables()
#         self.gradients_explore = tf.gradients(self.explore_policy_loss,self.tvars_explore)
        
        self.addBaseline()
        
#         self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=1, size = 16,output_activation=None)
#         self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
#         baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
#         self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)
        
#         #Encoder LSTM
        self.LSTMEncoder()
        
        
        #decoder weights
        #self.d_W1 = self.Decoder(scope = "W1", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W2 = self.Decoder(scope = "W2", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*action_dim)
        self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.observation_dim*self.action_dim)
        
        #self.d_W1 = ((self.d_W1 - (tf.reduce_max(self.d_W1) + tf.reduce_min(self.d_W1))/2)/(tf.reduce_max(self.d_W1) - tf.reduce_min(self.d_W1)))*2 
        #self.d_W2 = ((self.d_W2 - (tf.reduce_max(self.d_W2) + tf.reduce_min(self.d_W2))/2)/(tf.reduce_max(self.d_W2) - tf.reduce_min(self.d_W2)))*2 
        #self.d_W3 = ((self.d_W3 - (tf.reduce_max(self.d_W3) + tf.reduce_min(self.d_W3))/2)/(tf.reduce_max(self.d_W3) - tf.reduce_min(self.d_W3)))*2 
        
        #self.d_W1 = tf.reshape(self.d_W1, [self.observation_dim, self.decoder_len])
        #self.d_W2 = tf.reshape(self.d_W2, [self.observation_dim, self.decoder_len])
        #self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        self.d_W3 = tf.reshape(self.d_W3, [self.observation_dim, self.action_dim])
        
        # decoder output bias
        #self.d_B1 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B1"), [self.decoder_len])
        #self.d_B2 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B2"), [self.decoder_len])
        self.d_B3 = tf.reshape(self.Decoder(decoder_out_dim = self.action_dim, scope = "B3"), [self.action_dim])
        #self.d_B1 = ((self.d_B1 - (tf.reduce_max(self.d_B1) + tf.reduce_min(self.d_B1))/2)/(tf.reduce_max(self.d_B1) - tf.reduce_min(self.d_B1)))*2 
        #self.d_B2 = ((self.d_B2 - (tf.reduce_max(self.d_B2) + tf.reduce_min(self.d_B2))/2)/(tf.reduce_max(self.d_B2) - tf.reduce_min(self.d_B2)))*2 
        
        
        # exploit policy
        self.build_policy_exploit()
        #self.d = [self.d_W1, self.d_B1, self.d_W2, self.d_B2, self.d_W3, self.d_B3]
        self.exploit_policy_loss = -tf.reduce_sum(self.exploit_logprob * self.advantage_placeholder_exploit)
        self.d = [self.d_W3, self.d_B3]
        self.gradients_exploit = tf.gradients(self.exploit_policy_loss,self.d)
        
        # train encoder and decoder
        adam_optimizer_exploit =  tf.train.AdamOptimizer(self.lr*0.3)
        self.output_train_op = adam_optimizer_exploit.minimize(self.exploit_policy_loss)
        # train original network
        adam_optimizer_explore = tf.train.AdamOptimizer(self.lr)
        self.input_train_op = adam_optimizer_explore.minimize(self.explore_policy_loss)

    
    def initialize(self):
        self.ConstructGraph()
        # create tf session
        self.sess = tf.Session()
        
        # initiliaze all variables
        init = tf.global_variables_initializer()
        self.sess.run(init)
    
    def train_step(self): 
        # sample num_traj*num_MDPs
        for i in range(self.num_mdps):
            explore_paths, explore_rewards = self.sample_paths_explore(self.env,i)
            observations_explore = np.concatenate([path["observation"] for path in explore_paths])
            new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
            new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
            actions_explore = np.concatenate([path["action"] for path in explore_paths])
            rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
            returns_explore = self.get_returns(explore_paths, explore = True)
            
#             vtargs, vpreds, advantages_explore = [], [], []
#             for path in explore_paths:
#                 rew_t = path["new_rewards"]
#                 return_t = utils.discount(rew_t, self.gamma)
#                 vpred_t = self.vf.predict(path["new_obs"])
#                 adv_t = return_t - vpred_t
#                 advantages_explore.append(adv_t)
#                 vtargs.append(return_t)
#                 vpreds.append(vpred_t)
                
                
#             #returns_explore = self.get_returns(explore_paths, explore = True)
#             advantages_explore = np.concatenate(advantages_explore)
#             advantages_explore = (advantages_explore - advantages_explore.mean()) / (advantages_explore.std() + 1e-8)
#             vtarg_n = np.concatenate(vtargs)
#             vpred_n = np.concatenate(vpreds)
#             #update baseline
#             #print(advantages_explore.shape,new_observations_explore.shape)
#             self.vf.fit(new_observations_explore, vtarg_n)
            
#             surr_loss, oldmean_na, oldlogstd_a = self.policyfn.update_policy(
#                     new_observations_explore, new_actions_explore, advantages_explore, self.lr)
            
            print("average reward explore: MDP", i, np.mean(explore_rewards), len(explore_rewards))
            
            #print(returns_explore)
#             print("average reward explore: MDP", i, np.sum(explore_rewards)/num_traj, len(explore_rewards))

            baseline_explore = self.sess.run(self.baseline, {self.observation_placeholder_explore:new_observations_explore})
            adv = returns_explore - np.squeeze(baseline_explore)
            advantages_explore = (adv - np.mean(adv))/(np.std(adv) + 1e-12)


            # update the baseline

            self.sess.run(self.update_baseline_op, {self.observation_placeholder_explore:new_observations_explore, 
                                           self.baseline_target_placeholder : returns_explore})

            # calculate explore gradients
    #         grads_explore = self.sess.run(self.gradients_explore, feed_dict={
    #                     self.observation_placeholder_explore : observations_explore,
    #                     self.action_placeholder_explore : actions_explore,
    #                     self.advantage_placeholder_explore : returns_explore})
            #print("explore",grads_explore )
            # form trajectory matrix

            M = np.array(self.stack_trajectories(explore_paths))


            #encoder LSTM
            Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
            Z = np.reshape(Z,[1,len(Z)])
#             #print("Z",Z)

#             # sample paths
#             tvars = tf.trainable_variables()
#             tvars_vals = self.sess.run(tvars[-5:-1])
#             #print(tvars_vals)
            for j in range(0):
                exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,i)
                # get observations, actions and rewards
                observations_exploit = np.concatenate([path["observation"] for path in exploit_paths])
                actions_exploit = np.concatenate([path["action"] for path in exploit_paths])
                rewards_exploit = np.concatenate([path["reward"] for path in exploit_paths])
                returns_exploit = self.get_returns(exploit_paths)
                print("average reward exploit: MDP", i, np.mean(exploit_rewards), len(exploit_rewards))


#             # exploit grads
#     #         grads_exploit = self.sess.run(self.gradients_exploit,feed_dict={
#     #                     self.observation_placeholder_exploit : observations_exploit,
#     #                     self.action_placeholder_exploit : actions_exploit,
#     #                     self.advantage_placeholder_exploit : returns_exploit,
#     #                     self.decoder_input_placeholder: Z})

            #train encoder and decoder network
                self.sess.run(self.output_train_op, feed_dict={
                                self.observation_placeholder_exploit : observations_exploit,
                                self.action_placeholder_exploit : actions_exploit,
                                self.advantage_placeholder_exploit : returns_exploit,
                                self.decoder_input_placeholder: Z})

            # find advantage for input network
    #         advantage_explore = 0
    #         for i in range(len(grads_exploit)):
    #             l1 = grads_exploit[i]
    #             l2 = grads_explore[i]
    #             advantage_explore = advantage_explore + np.matmul(l1.flatten(), l2.flatten())

            #train input policy
            self.sess.run(self.input_train_op, feed_dict={
                            self.observation_placeholder_explore : new_observations_explore,
                            self.action_placeholder_explore : new_actions_explore,
                            self.advantage_placeholder_explore : returns_explore})
    
    def test(self):
        explore_paths, explore_rewards = self.sample_paths_explore(self.env,20*np.random.rand())
        print("gravity: ",self.sim.model.opt.gravity )
        M = self.stack_trajectories(explore_paths)
        Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
        Z = np.reshape(Z,[1,len(Z)])
        # sample paths
        exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,20*np.random.rand())
        print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
        
    def train(self):
        self.initialize()
        num_epochs = 200
        for epoch in range(num_epochs):
            #self.num_traj = 20
            print("epoch number: ", epoch)
            self.train_step()
#             print("evaluating on new MDPs")
#             self.num_traj = 400
#             self.test()

            
            

In [ ]:
a = MetaLearner(env, max_ep_len, num_traj, latent_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [ ]:
a.train()

epoch number:  0
average reward explore: MDP 0 -5583.598018273133 3
average reward explore: MDP 1 -4809.484868915452 3
average reward explore: MDP 2 -5052.846445813212 3
average reward explore: MDP 3 -4319.391044290912 3
average reward explore: MDP 4 -4319.466598941304 3
average reward explore: MDP 5 -4082.2975601553658 3
average reward explore: MDP 6 -3360.033697006263 3
average reward explore: MDP 7 -3469.6460017439917 3
average reward explore: MDP 8 -2793.539014690749 3
average reward explore: MDP 9 -2902.800488821465 3
epoch number:  1
average reward explore: MDP 0 -2811.716964443236 3
average reward explore: MDP 1 -2805.0019011205436 3
average reward explore: MDP 2 -2833.3712956085637 3
average reward explore: MDP 3 -2484.0084286247875 3
average reward explore: MDP 4 -2762.3671141181526 3
average reward explore: MDP 5 -2805.3309695782955 3
average reward explore: MDP 6 -2825.4149105311176 3
average reward explore: MDP 7 -2569.8552030291416 3
average reward explore: MDP 8 -2900.026

average reward explore: MDP 4 -1826.0604122986742 3
average reward explore: MDP 5 -1546.386363674209 3
average reward explore: MDP 6 -1672.3151797184728 3
average reward explore: MDP 7 -1802.0984585680792 3
average reward explore: MDP 8 -1537.5080173912918 3
average reward explore: MDP 9 -1847.3424046732473 3
epoch number:  16
average reward explore: MDP 0 -1518.7034342635545 3
average reward explore: MDP 1 -1538.667901087536 3
average reward explore: MDP 2 -1593.1648463486981 3
average reward explore: MDP 3 -1643.007981491491 3
average reward explore: MDP 4 -1487.1683376686399 3
average reward explore: MDP 5 -1566.0263053994931 3
average reward explore: MDP 6 -1906.5072385402518 3
average reward explore: MDP 7 -1557.702180744937 3
average reward explore: MDP 8 -1695.9774074552006 3
average reward explore: MDP 9 -1828.0057917363554 3
epoch number:  17
average reward explore: MDP 0 -1491.8090849397984 3
average reward explore: MDP 1 -1577.3338122057776 3
average reward explore: MDP 2 -1

average reward explore: MDP 8 -957.7031939513639 3
average reward explore: MDP 9 -936.28863295868 3
epoch number:  31
average reward explore: MDP 0 -1163.3645766083791 3
average reward explore: MDP 1 -1089.9270415439887 3
average reward explore: MDP 2 -1170.5760214093295 3
average reward explore: MDP 3 -903.8212881217684 3
average reward explore: MDP 4 -953.2166323120218 3
average reward explore: MDP 5 -1078.7706610215116 3
average reward explore: MDP 6 -960.1257415873464 3
average reward explore: MDP 7 -1087.131039748478 3
average reward explore: MDP 8 -1072.8576862173786 3
average reward explore: MDP 9 -1038.2061491164777 3
epoch number:  32
average reward explore: MDP 0 -1031.4343127828072 3
average reward explore: MDP 1 -1224.3072825629897 3
average reward explore: MDP 2 -968.9139080161855 3
average reward explore: MDP 3 -983.2912410336448 3
average reward explore: MDP 4 -1018.2732000645166 3
average reward explore: MDP 5 -1103.1050390110668 3
average reward explore: MDP 6 -1026.67

average reward explore: MDP 3 -745.1757309710111 3
average reward explore: MDP 4 -740.8456646609608 3
average reward explore: MDP 5 -682.8863603943342 3
average reward explore: MDP 6 -621.6983424070801 3
average reward explore: MDP 7 -702.106861466635 3
average reward explore: MDP 8 -715.570558367246 3
average reward explore: MDP 9 -759.7638027534562 3
epoch number:  47
average reward explore: MDP 0 -759.9707099552478 3
average reward explore: MDP 1 -690.5168044495318 3
average reward explore: MDP 2 -839.4621635207321 3
average reward explore: MDP 3 -633.4802516673328 3
average reward explore: MDP 4 -672.4700706524433 3


In [ ]:
for i in range(20):
    explore_paths, explore_rewards = a.sample_paths_explore(a.env,200*np.random.rand())
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore:", np.mean(explore_rewards))
    M = a.stack_trajectories(explore_paths)
    Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
    Z = np.reshape(Z,[1,len(Z)])
    exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)
    print("average reward exploit", np.mean(exploit_rewards), len(exploit_rewards))

In [ ]:
a.lr = 3e-3
num_epochs = 50
for epoch in range(num_epochs):
        print("epoch number: ", epoch)
        a.train_step()

In [32]:
a.num_traj = 800

In [ ]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,1)

In [124]:
print("average reward explore:", np.mean(explore_rewards), len(explore_rewards))
M = a.stack_trajectories(explore_paths)

average reward explore: 151.57677928797034 400


In [125]:
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })

In [126]:
Z = np.reshape(Z,[1,len(Z)])

In [127]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)

In [128]:
print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit 18.577710931432883 400


In [122]:
a.num_traj = 800

In [83]:
for i in range(10):
    maxrev2 = 0
    maxrev1 = 0
    traj = np.random.rand()*(i+1)
    
    for i in range(4):
        explore_paths, explore_rewards = a.sample_paths_explore(a.env,traj)
        #print("average reward explore:", np.mean(explore_rewards))
        M = a.stack_trajectories(explore_paths)
        Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
        Z = np.reshape(Z,[1,len(Z)])
        exploit_paths, explore_rewards = a.sample_paths_exploit(a.env,Z,traj)
        if(np.mean(explore_rewards) > maxrev1):
            maxrev1 = np.mean(explore_rewards) / num_traj
        if(np.mean(exploit_rewards)>maxrev2):
            maxrev2 = np.mean(exploit_rewards) / num_traj
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore", maxrev1)
    print("average reward exploit", maxrev2)
    

gravity: [ 0.         0.        -5.4325712]
average reward explore 3.5619071147766848
average reward exploit 2.7301699712255383


KeyboardInterrupt: 

In [90]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,5)
observations_explore = np.concatenate([path["observation"] for path in explore_paths])
new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
actions_explore = np.concatenate([path["action"] for path in explore_paths])
rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
returns_explore = a.get_returns(explore_paths, explore = True)

In [91]:
M = np.array(a.stack_trajectories(explore_paths))
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
Z = np.reshape(Z,[1,len(Z)])

In [92]:
 print("average reward explore: MDP", np.sum(explore_rewards)/num_traj, len(explore_rewards))

average reward explore: MDP 20488.85105031593 800


In [93]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,5)

In [94]:
print("average reward exploit: MDP", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit: MDP 644.4290518284578 400


In [ ]:
def randomize_pendulum(env):
    m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])
    l = config['pendulum_len_min'] + np.random.rand()*(config['pendulum_len_max'] - config['pendulum_len_min'])
    env.m = m
    env.l = l

In [ ]:
def randomize_hopper(env):
    ts = config['torso_min'] + np.random.rand()*(config['torso_max'] - config['torso_min'])
    f = config['friction_min'] + np.random.rand()*(config['friction_max'] - config['friction_min'])
    
    env.friction = f
    env.torso_size = ts
    env.apply_env_modifications()

In [ ]:
import yaml
cfg_filename = 'hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
env = Randomizer(gym.make('Hopper-v2'), randomize_hopper)
env.unwrapped.__dict__.keys()
env.reset()

In [ ]:
m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])

In [ ]:
config

In [ ]:
env_id = 'Hopper-v2'
e = gym.make(env_id)

In [ ]:
e.unwrapped.__dict__.keys()

In [ ]:
e.unwrapped.apply_env_modifications()

In [7]:
a.sim.model.opt

In [29]:
exploit_rewards

[112.13464748855134,
 107.88303586312765,
 186.02669267001477,
 135.2581011205201,
 79.13080731610503,
 121.000075027127,
 173.12633008893275,
 183.049128023303,
 178.67252211302042,
 87.14624506369364,
 194.95350061967267,
 99.25651092576477,
 71.49508077704641,
 88.63821085625852,
 102.13457423812075,
 158.97240001687823,
 116.1121244537115,
 107.73547738516186,
 139.42624826594056,
 66.64122421143784,
 180.7967956127941,
 81.9917085465781,
 86.7164580284039,
 216.9255537144621,
 196.23999804809637,
 106.97407952338774,
 211.77799925827094,
 138.13634311633172,
 146.8420615449942,
 128.17126276654236,
 74.77819434837676,
 183.75160687642273,
 72.78452895798674,
 120.41099605928771,
 101.5334367194092,
 168.35981035001126,
 69.62303405123133,
 121.26107855071695,
 158.01986971006684,
 200.6322013111029,
 71.87886597554524,
 186.44100396410624,
 90.5373394288748,
 162.02200487464742,
 171.76682194626335,
 64.66835988088545,
 93.07526440100999,
 166.23306413321885,
 105.99062506542288,


In [31]:
len(exploit_rewards)

800